In [1]:
import os

import gzip
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
f = gzip.GzipFile(r"../../Data/rBergomiTrainSet.txt.gz", "r")
dat=np.load(f)
xx=dat[:,:4]
yy=dat[:,4:]
strikes=np.array([0.5,0.6,0.7,0.8,0.9,1.0,1.1,1.2,1.3,1.4,1.5 ])
maturities=np.array([0.1,0.3,0.6,0.9,1.2,1.5,1.8,2.0 ])

In [2]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler


x_train, x_test, y_train, y_test = train_test_split(
    xx, yy, test_size=0.15, random_state=42)

def append_and_expand(a,x,y):
    # use choose and where !
    n = len(x)*len(y)
    a_index = np.arange(len(a))%n
    
    
    x_index = a_index//len(y)
    y_index = a_index%len(y)
    
    x_added = np.choose(x_index,x.reshape(-1,1)).reshape(-1,1)
    y_added = np.choose(y_index,y.reshape(-1,1)).reshape(-1,1)
    
    return np.hstack([a,x_added,y_added])

y_train,y_test = y_train.reshape(-1,8,11),y_test.reshape(-1,8,11)
x_train,x_test = np.repeat(x_train, 8*11,axis=0),np.repeat(x_test, 8*11,axis=0)

x_train,x_test=append_and_expand(x_train,maturities,strikes),append_and_expand(x_test,maturities,strikes)
y_train,y_test = y_train.reshape(-1).reshape(-1,1), y_test.reshape(-1).reshape(-1,1)

In [3]:
import torch
import torch.nn as nn

scale_y=  StandardScaler()

def ytransform(y_train,y_test):
    return [scale_y.fit_transform(y_train),scale_y.transform(y_test)]

def yinversetransform(y):
    return scale_y.inverse_transform(y)

# Upper and lower bounds used in the training set
ub=np.array([0.16,4,-0.1,0.5,2.0,1.5])
lb=np.array([0.01,0.3,-0.95,0.025,0.1,0.5])

def myscale(x):
    return (x - (ub+lb)*0.5)*2/(ub-lb)
def myinverse(x):
    return x*(ub-lb)*0.5+(ub+lb)*0.5

x_train_transform = myscale(x_train)
x_test_transform = myscale(x_test)
[y_train_transform,y_test_transform] = ytransform(y_train,y_test)

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
print(f"device is {device}")

train_dataset = torch.utils.data.TensorDataset(torch.from_numpy(x_train_transform).to(device=device),
                                               torch.from_numpy(y_train_transform).to(device=device))
test_dataset = torch.utils.data.TensorDataset(torch.from_numpy(x_test_transform).to(device=device),
                                              torch.from_numpy(y_test_transform).to(device=device))


train_data = (torch.from_numpy(x_train_transform).to(device=device),torch.from_numpy(y_train_transform).to(device=device))
test_data = (torch.from_numpy(x_test_transform).to(device=device),torch.from_numpy(y_test_transform).to(device=device))

data_loader = torch.utils.data.DataLoader(train_dataset,batch_size =32,shuffle=True)

device is cuda


In [4]:
import sys
sys.path.append('../../')  # Add the parent directory to the Python path

from torch_NN.nn import KAN

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
# hyperparas = {'input_dim':6,'hidden_dim':32,'hidden_nums':2,'output_dim':1,'block_layer_nums':2}

layers_list_for_KAN = [x_train.shape[-1],32,1]

model = KAN(layers_hidden=layers_list_for_KAN).to(device=device,dtype=torch.float64)
loss_MSE = nn.MSELoss()
optim_Adam = torch.optim.Adam(model.parameters(),lr= 0.001)

In [5]:
from torch_NN.train import train_model

train_model(loss_MSE,optim_Adam,model,data_loader,train_data,test_data,3,test=False)

-----------------------Epoch: 1----------------------------------
Batch: 0%, train loss is: 0.6596936951828097
Batch: 1%, train loss is: 0.011802204690760317
Batch: 2%, train loss is: 0.0042651602446630195
Batch: 3%, train loss is: 0.005274703318624546
Batch: 4%, train loss is: 0.004619155679744886
Batch: 5%, train loss is: 0.0016648206648426497
Batch: 6%, train loss is: 0.0007955268166093207
Batch: 7%, train loss is: 0.0010250921326635447
Batch: 8%, train loss is: 0.0011711440730029056
Batch: 9%, train loss is: 0.0017959549103175052
Batch: 10%, train loss is: 0.0009429514919153238
Batch: 11%, train loss is: 0.0011772051014358482
Batch: 12%, train loss is: 0.0011066854163776319
Batch: 13%, train loss is: 0.0007214779968843658
Batch: 14%, train loss is: 0.0006821473209847559
Batch: 15%, train loss is: 0.0009320434284458842
Batch: 16%, train loss is: 0.0005209919907180281
Batch: 17%, train loss is: 0.0004279553306613944
Batch: 18%, train loss is: 0.0006699368598777972
Batch: 19%, train l

In [7]:
for param_group in optim_Adam.param_groups:
    param_group['lr'] = 0.0001

train_model(loss_MSE,optim_Adam,model,data_loader,train_data,test_data,7,test=False)

-----------------------Epoch: 1----------------------------------
Batch: 0%, train loss is: 0.00016478871484257616
Batch: 1%, train loss is: 0.00018496078120853942
Batch: 2%, train loss is: 0.00011160217828781228
Batch: 3%, train loss is: 0.00014543215718357602
Batch: 4%, train loss is: 0.00010805820341015608
Batch: 5%, train loss is: 8.634524504346583e-05
Batch: 6%, train loss is: 0.00012615718561986893
Batch: 7%, train loss is: 0.0004957112561547954
Batch: 8%, train loss is: 7.078870265320839e-05
Batch: 9%, train loss is: 0.00012311680388680957
Batch: 10%, train loss is: 0.0002547191653876898
Batch: 11%, train loss is: 0.00029670150444178426
Batch: 12%, train loss is: 9.329084571421873e-05
Batch: 13%, train loss is: 0.0001222879176347357
Batch: 14%, train loss is: 0.00010786899171185985
Batch: 15%, train loss is: 8.241233606670295e-05
Batch: 16%, train loss is: 0.0001617947973773173
Batch: 17%, train loss is: 0.0002716651833331634
Batch: 18%, train loss is: 6.661243300234794e-05
Batc

In [8]:
torch.save(model.state_dict(),'../restruct_data_results/rflatBergomi_pointwise88_KAN.pth')